<a href="http://landlab.github.io"><img style="float: left" src="../../landlab_header.png"></a>

# Landlab Landslide Component Demonstration with a Synthetic Grid

**Two Hydrologic Forcing Options (Depth to Groundwater and Recharge) are compared using default parameters and Uniform Distribution**


<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>

## 0.  Software Setup and Preparation

To run this notebook, we must import several libraries.
The hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading, and creation.  Additional libraries support the functions of Landlab. 

### Import Landlab and other Python utilities

In [1]:
import numpy as np
from landlab import RasterModelGrid
from landlab.components.landslides import LandslideProbability
import landlab.plot.landslides.plot_landslides as pl
import landlab.grid.landslide_unitgrid as lgd
import tests.components.landslides.test_landslide_probability as test

ModuleNotFoundError: No module named 'landlab'

### Landslide Component Definitions
#### key variables and functions in `test_landslide_probability.py`

`grid` is the Landlab raster model grid initialization

`ls_grid` is a `grid`  with landslide parameters added to nodes
The input fields (see input_var_names) on the node are initialized in this step.

`ls_grid` is input to `LandslideProbability()` to add model parameters to define the size of the Monte Carlo method with hydrologic forcings (two options: recharge OR depth to water table) and statistical distribution (four options) and  for the landslide model. A distribution of hydrologic forcing is used to calculate the probability of lansdlides in the Factor of Safety equation; the mean hydrologic forcing  per node has been added as a model output (in addition to relative wetness, probability of saturation, and probability of failure). 

`ls_prob` is the output of `LandslideProbability()` and considered a model instance (or model initialization).

`ls_prob` is the input to `calculate_landslide_probability()`. Model results are saved in the same input dictionary by adding output fields.

`ls_prob` is both the input and output of  `calculate_landslide_probability()` and should be saved, or recalculated with a clean `ls_grid` to ensure variables are saved as expected (not overwritten) for each change of parameters and model calculation.

The output fields (see output_var_names) on the node are initialized to zero, and contain the model results. 

## 1. Create a synthetic grid with landslide unit_default data fields

**1.1 Use the unit test to build a synthetic grid investigate features of the grid.** 

The total number of nodes is limited to 20, which is fast for computers and easy for humans.

In [ ]:
shape = (5, 4)
spacing = (10e0, 10e0)
coordinates = (0.0, 0.0)
grid = RasterModelGrid(shape,spacing,coordinates)

print('The synthetic grid `grid` is a Landlab RasterModelGrid: {value}. '.format(value=grid))
grid

We can see how many core nodes we use in the unit grid (i.e., the nodes that calculations are performed on) in the center of the grid...not the edges, which are 'closed nodes'.  The spacing between nodes is equivalent to the grid cell edge length.  Nodes are the centroid of each grid.

In [ ]:
gridnum = grid.number_of_nodes
print('The synthetic grid has {value} total grid cells (nodes), including the boundary nodes.'.format(value=grid.number_of_nodes)) 
print('The synthetic grid has {value} m grid cell size (space between nodes).'.format(value=grid.dx)) 
grid_cnum = grid.number_of_core_nodes
print("Core nodes are the center of a Landlab model grid.")
print("Landslide calculations are made only on {value} core nodes.".format(value=grid_cnum))
print("This is the array of Landlab node values used to index the core nodes (dictionary key)")
grid.core_nodes

In [ ]:
#grid?

To read more documentation about this dataset type:

Use `RasterModelGrid?` OR `unitgrid?` OR uncomment cell above.

<hr>
<small>For more Introduction to Landlab’s Gridding Library, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/grid.html">https://landlab.readthedocs.io/en/latest/user_guide/grid.html</a></small>
<hr>

The Landlab grid variable we input to `LandslideProbability` is a `RasterModelGrid` data type we have named `grid`, it has inputs and outputs on each node, but only calculated on the core nodes.

In [ ]:
 sorted(LandslideProbability.input_var_names)

## Write a function to build a landslide component unit grid

In [ ]:
ls_prob, ls_grid = lgd.build_landslide_unitgrid(shape,spacing,coordinates,'depth')

In [ ]:
print("Landslide Component Input Variables are on the grid/node")
ls_prob.input_var_names

In [ ]:
print("Landslide Component Output Variables at each grid/node")
ls_prob.output_var_names

In [ ]:
LandslideProbability.var_help('soil__mean_watertable_depth')

In [ ]:
LandslideProbability.var_help('soil__mean_recharge')

**1.2 Investigate the range of default parameters on each node of the grid.**

The test builds a grid with a default synthetic range of min/max parameter inputs used in the unit test. The function `get_default_nodevalues()` creates a variable from the default values.  Since we haven't calculated the landslide probability yet, only initialized the inputs, the output values are initialized to zeros. 

In [ ]:
test_unit_defaultpars=test.get_default_nodevalues()
test_unit_defaultpars

Run the next cells to view DEFAULT soil parameters on the node.(optional)

In [ ]:
value_list = ['topographic__slope']
pl.print_list_nodevalues(ls_prob, value_list)

In [ ]:
#value_list = ['soil__transmissivity']
#pl.print_list_nodevalues(ls_prob, value_list)

Run the next cells to CHANGE DEFAULT soil parameters on the node.(optional)

In [ ]:
##Transmissivity
#grid.at_node["soil__transmissivity"] = np.sort(np.random.uniform(0.1,100,gridnum).astype(float))    
#reverse order: lowest Transmissivity value are in upper right of synthethic grid
#grid['node']['soil__transmissivity'] = grid['node']['soil__transmissivity'][::-1]  
       

Add a field for dimensions of recharge in millimeters. Default input is in meters.

In [ ]:
recharge_mm_per_m=1000
ls_grid.at_node['soil__mean_recharge_mm']=recharge_mm_per_m * ls_grid.at_node['soil__mean_recharge']

In [ ]:
value_list = ['soil__transmissivity']
pl.print_list_nodevalues(ls_prob, value_list)

In [ ]:
value_list = ['soil__saturated_hydraulic_conductivity','soil__transmissivity','soil__thickness']
pl.print_list_nodevalues(ls_prob, value_list)

In [ ]:
#value_list = ['soil__minimum_total_cohesion','soil__maximum_total_cohesion','soil__mode_total_cohesion']
#pl.print_list_nodevalues(ls_grid, value_list)

### 1.13 Make Landslide Unit Grid Maps

The plot function `plot_landslide_4variables()` uses the Landlab plotting function `imshow_grid()` and creates subplots for any 4 variables on a Landlab grid.

Let's see what one of these fields (slope) looks like on the grid of 10 x 10 m grid cells. 

In [ ]:
ls_grid.at_node['topographic__slope']


In [ ]:
fields=['topographic__slope','soil__thickness','topographic__specific_contributing_area','soil__transmissivity']
maxval=[ls_grid.at_node['topographic__slope'].max(),
        ls_grid.at_node['soil__thickness'].max(),
        ls_grid.at_node['topographic__specific_contributing_area'].max(),
        ls_grid.at_node['soil__transmissivity'].max(),]
subtitles=['Unit Slope','Unit Soil Thickness','Unit Contributing Area','Unit Transmissivity']
colors=['terrain','copper_r','YlGnBu','PuBu']
scalelabel=['Slope',' meters','meters','m/day']

pl.plot_landslide_4variables([221,222,223,224],
                          ls_grid,ls_grid,ls_grid,ls_grid,
                          fields,maxval,
                          subtitles,colors,scalelabel)

Can't get enough nitty gritty landslide component details?  

Uncomment the box below to review ALL the details of what each variable represents.  
(Close the window with the upper right [x] after viewing.)

In [ ]:
#?LandslideProbability

 ### 2.3. Recharge and Depth Options - 'uniform' distribution 

In this option, we'll apply a uniform distribution of recharge and depth based on minimum and maximum recharge and depth values.  We will then instantiate the LandslideProbability component with these parameters, as well as the grid and number of iterations we specified above.  Instantiate creates an instance of a class.  (For example, the _iPhone_ is a class and _each phone_ is an instance.)

No outputs are generated by these commands as they are setting up the recharge and depth and instantiating the component.


In [ ]:
LS_prob1_r,grid_r1 = lgd.build_landslide_unitgrid(shape,spacing,coordinates,'recharge')
LS_prob1_d,grid_d1 = lgd.build_landslide_unitgrid(shape,spacing,coordinates,'depth')

print("Recharge Hydrologic Forcing")
LS_prob1_r.calculate_landslide_probability()
test.print_failure_nodevalues(LS_prob1_R)
print("")
print("Depth Hydrologic Forcing")
LS_prob1_d.calculate_landslide_probability()
test.print_failure_nodevalues(LS_prob1_d)



In [ ]:
print("Uniform Recharge Synthetic core node 6 = {value}".format(value=LS_prob1_r.grid.at_node["landslide__probability_of_failure"][5]))
print("Uniform Recharge Synthetic  core node 10 = {value}".format(value=LS_prob1_r.grid.at_node["landslide__probability_of_failure"][9]))
print("Uniform Depth Synthetic  core node 6 = {value}".format(value=LS_prob1_d.grid.at_node["landslide__probability_of_failure"][5]))
print("Uniform Depth Synthetic  core node 10 = {value}".format(value=LS_prob1_d.grid.at_node["landslide__probability_of_failure"][9]))

In [ ]:
# Commercial Break - Adventures in printing landslide outputs in Python - 
# brought to you by your friends at Waterhackweek - waterhackweek.github.io
# printing options (optional options)
print("This is an example of using `print_failure_nodevalues()` function to print the failure outputs for any Landslide grid.")
test.print_failure_nodevalues(LS_prob1_d)
print("This is an example of using `print_list_nodevalues()` function to print the variables on the node limited to those in the list")
test.print_list_nodevalues(LS_prob1_r,["landslide__probability_of_failure"])
print("Unit test core node 6 = {value}".format(value=LS_prob1_d.grid.at_node["landslide__probability_of_failure"][5]))
print("Unit test core node 10 = {value}".format(value=LS_prob1_d.grid.at_node["landslide__probability_of_failure"][9]))

In [ ]:
#Default relative wetness calculated with unitgrid input values 
#this is used to calculate the max and min Depth and Recharge for plotting
relative_wetness = 0.75


In [ ]:
#Main Plot inputs
pgrid = LS_prob1_d.grid
[unit_R,unit_D]=test.scenario_unit_explorer(relative_wetness,pgrid) #set max
#Landslide plot inputs for Uniform Depth forcing
fields=['soil__probability_of_saturation','soil__mean_relative_wetness','soil__mean_watertable_depth','landslide__probability_of_failure']
maxval=[1,1, unit_D[1],1,]
colors=['YlGnBu','YlGnBu','YlGnBu','OrRd']
scalelabel=['Probability',' Wetness','Depth (m)','Probability']
subtitle=['Prob.of Saturation','Relative Wetness','Mean Depth to Water Table','Probability of Failure']
smashtitle=subtitle
ptitle = "Uniform-Depth %s"
for s in subtitle: smashtitle[subtitle.index(s)]="".join(ptitle % (s))
    
pl.plot_landslide_4variables([221,222,223,224],
                          pgrid,pgrid,pgrid,pgrid,
                          fields,maxval,
                          smashtitle,colors,scalelabel)

### Uniform-Recharge
Recharge outputs are in meters (m), here we convert the recharge to mm/day and add to the grid. 

In [ ]:
print(unit_R)
recharge_mm_per_m=1000
LS_prob1_r.grid.at_node['soil__mean_recharge_mm']=recharge_mm_per_m * LS_prob1_r.grid.at_node['soil__mean_recharge']

In [ ]:
LS_prob1_d.grid.at_node['soil__mean_recharge']

In [ ]:
LS_prob1_r.grid.at_node['soil__mean_recharge']

In [ ]:
pgrid = LS_prob1_r.grid
[unit_R,unit_D]=test.scenario_unit_explorer(relative_wetness,pgrid)
fields=['soil__probability_of_saturation','soil__mean_relative_wetness','soil__mean_recharge_mm','landslide__probability_of_failure']
maxval=[1,1, unit_R[1],1,]
colors=['YlGnBu','YlGnBu','YlGnBu','OrRd']
scalelabel=['Probability',' Wetness','Recharge (mm/day)','Probability']
subtitle=['Prob.of Saturation','Relative Wetness','Mean Recharge','Probability of Failure']
smashtitle=subtitle
ptitle = "Uniform-Recharge %s"
for s in subtitle: smashtitle[subtitle.index(s)]="".join(ptitle % (s))
    
pl.plot_landslide_4variables([221,222,223,224],
                          pgrid,pgrid,pgrid,pgrid,
                          fields,maxval,
                          smashtitle,colors,scalelabel)

## Citing Landlab Landslide Component

**If you use any portion of Landlab, you must cite the following papers:**

Hobley, D. E. J., Adams, J. M., Nudurupati, S. S., Hutton, E. W. H., Gasparini, N. M., Istanbulluoglu, E. and Tucker, G. E., 2017, Creative computing with Landlab: an open-source toolkit for building, coupling, and exploring two-dimensional numerical models of Earth-surface dynamics, Earth Surface Dynamics, 5, p 21-46, 10.5194/esurf-5-21-2017.

Barnhart, K. R., Hutton, E. W. H., Tucker, G. E., Gasparini, N. M., Istanbulluoglu, E., Hobley, D. E. J., Lyons, N. J., Mouchene, M., Nudurupati, S. S., Adams, J. M., and Bandaragoda, C.: Short communication: Landlab v2.0: A software package for Earth surface dynamics, Earth Surf. Dynam. Discuss., https://doi.org/10.5194/esurf-2020-12, in review, 2020.
                
**If you use any portion of the Landlab Landslide Component, you must cite the following paper:**
Strauch, R., Istanbulluoglu, E., Nudurupati, S.S., Bandaragoda, C., Gasparini, N.M. and Tucker, G.E., 2018. A hydroclimatological approach to predicting regional landslide probability using Landlab. Earth Surface Dynamics, 6(1), pp.49-49.

<img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:150px;padding:20px">   


The development of this Notebook has been made possible by the Landlab project funded by the National Science Foundation (OAC 1450338 to N. Gasparini, OAC 1450409 to G. Tucker, OAC 1450412 to E. Istanbulluoglu).

We acknowledge that this research takes place in the ancestral homelands of Tribes, Bands, and First Nations, including most notably the Upper Skagit Indian Tribe, Sauk-Suiattle Indian Tribe, Swinomish Indian Tribal Community, Samish Indian Nation, Nooksack Tribe, Lummi Nation, Stó:lō Nation, Nlaka’pamux Nation, Colville Confederated Tribes, Syilx/Okanagan Nation.  The University of Washington acknowledges the Coast Salish peoples of this land, the land which touches the shared waters of all tribes and bands within the Suquamish, Tulalip and Muckleshoot nations. 


<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>